In [1]:
!pip uninstall -y pillow
!pip install pillow==10.2.0 --force-reinstall --no-cache-dir


Found existing installation: pillow 11.3.0
Uninstalling pillow-11.3.0:
  Successfully uninstalled pillow-11.3.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 81.8 MB/s eta 0:00:00


In [1]:
!pip install facenet-pytorch

import os
import numpy as np
import torch
from facenet_pytorch import InceptionResnetV1
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from PIL import Image
import torchvision.transforms as transforms
import joblib
from collections import Counter


In [2]:
import kagglehub
import tarfile
import os


path = kagglehub.dataset_download("atulanandjha/lfwpeople")
print("✅ Dataset downloaded to:", path)

tgz_path = os.path.join(path, "lfw-funneled.tgz")

extract_path = "/kaggle/working/lfw_funneled"

with tarfile.open(tgz_path, "r:gz") as tar:
    tar.extractall(path=extract_path)

print("✅ Extraction done!")
print("📂 Number of folders:", len(os.listdir(extract_path)))
print("📂 First 5 people:", os.listdir(extract_path)[:5])


100%|██████████| 232M/232M [00:02<00:00, 106MB/s]

Extracting files...


✅ Dataset downloaded to: /root/.cache/kagglehub/datasets/atulanandjha/lfwpeople/versions/3


/tmp/ipython-input-3646392181.py:17: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
  tar.extractall(path=extract_path)


✅ Extraction done!
📂 Number of folders: 1
📂 First 5 people: ['lfw_funneled']


In [3]:
import torch
from facenet_pytorch import InceptionResnetV1
import torchvision.transforms as transforms
from PIL import Image
import numpy as np


# Use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("✅ Using device:", device)

# Load FaceNet
facenet = InceptionResnetV1(pretrained="vggface2").eval().to(device)

# Required transforms
transform = transforms.Compose([
    transforms.Resize((160, 160)),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

def get_embedding(image_pil):
    img_tensor = transform(image_pil).unsqueeze(0).to(device)
    with torch.no_grad():
        emb = facenet(img_tensor)
    return emb.squeeze(0).cpu().numpy()


✅ Using device: cuda


  0%|          | 0.00/107M [00:00<?, ?B/s]

In [4]:
X, y = [], []

lfw_root = os.path.join(extract_path, "lfw_funneled")

for person in os.listdir(lfw_root):
    person_folder = os.path.join(lfw_root, person)
    if not os.path.isdir(person_folder):
        continue

    for img_name in os.listdir(person_folder):
        img_path = os.path.join(person_folder, img_name)
        if os.path.isfile(img_path):
            try:
                img = Image.open(img_path).convert("RGB")
                emb = get_embedding(img)
                X.append(emb)
                y.append(person)
            except Exception as e:
                print("❌ Error with:", img_path, e)

X = np.array(X)
y = np.array(y)

print("✅ Number of embeddings:", X.shape)
print("✅ Number of people:", len(set(y)))


✅ Number of embeddings: (13233, 512)
✅ Number of people: 5749


In [5]:
import numpy as np
from collections import Counter
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import joblib


counts = Counter(y)
mask = np.array([counts[label] > 1 for label in y])

X_filtered = X[mask]
y_filtered = y[mask]

print("✅ After filtering:")
print("Number of images:", X_filtered.shape[0])
print("Number of people:", len(set(y_filtered)))


X_train, X_test, y_train, y_test = train_test_split(
    X_filtered, y_filtered, test_size=0.2, random_state=42, stratify=y_filtered
)

classifier = SVC(kernel="linear", probability=True)
classifier.fit(X_train, y_train)

joblib.dump(classifier, "classifier.pkl")
print("✅ Model trained and saved as classifier.pkl")


y_pred = classifier.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print(f"📊 Accuracy on test set: {acc*100:.2f}%")

print("\n📑 Classification Report (first 20 classes):")
print(classification_report(y_test, y_pred, labels=np.unique(y_test)[:20]))

counts = Counter(y)
mask = np.array([counts[label] > 1 for label in y])

X_filtered = X[mask]
y_filtered = y[mask]

print("✅ After filtering:")
print("Number of images:", X_filtered.shape[0])
print("Number of persons:", len(set(y_filtered)))



X_train, X_test, y_train, y_test = train_test_split(
    X_filtered, y_filtered, test_size=0.2, random_state=42, stratify=y_filtered
)


classifier = SVC(kernel="linear", probability=True)
classifier.fit(X_train, y_train)


joblib.dump(classifier, "classifier.pkl")
print("✅ Model trained and saved as classifier.pkl")


y_pred = classifier.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print(f"📊 Accuracy on test set: {acc*100:.2f}%")

print("\n📑 Classification Report (first 20 classes):")
print(classification_report(y_test, y_pred, labels=np.unique(y_test)[:20]))


✅ بعد التصفية:
عدد الصور: 9164
عدد الأشخاص: 1680
✅ Model trained and saved as classifier.pkl
📊 Accuracy on test set: 80.03%

📑 Classification Report (first 20 classes):
                              precision    recall  f1-score   support

               Aaron_Peirsol       0.50      1.00      0.67         1
         Abdel_Nasser_Assidi       0.00      0.00      0.00         1
              Abdoulaye_Wade       1.00      1.00      1.00         1
                    Abdullah       1.00      1.00      1.00         1
                Abdullah_Gul       0.57      1.00      0.73         4
         Abdullah_al-Attiyah       1.00      1.00      1.00         1
                Abel_Pacheco       1.00      1.00      1.00         1
Abid_Hamid_Mahmud_Al-Tikriti       0.00      0.00      0.00         1
                Adam_Sandler       0.50      1.00      0.67         1
              Adel_Al-Jubeir       0.00      0.00      0.00         1
       Adolfo_Aguilar_Zinser       0.00      0.00      0.00 

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


✅ Model trained and saved as classifier.pkl
📊 Accuracy on test set: 80.03%

📑 Classification Report (first 20 classes):
                              precision    recall  f1-score   support

               Aaron_Peirsol       0.50      1.00      0.67         1
         Abdel_Nasser_Assidi       0.00      0.00      0.00         1
              Abdoulaye_Wade       1.00      1.00      1.00         1
                    Abdullah       1.00      1.00      1.00         1
                Abdullah_Gul       0.57      1.00      0.73         4
         Abdullah_al-Attiyah       1.00      1.00      1.00         1
                Abel_Pacheco       1.00      1.00      1.00         1
Abid_Hamid_Mahmud_Al-Tikriti       0.00      0.00      0.00         1
                Adam_Sandler       0.50      1.00      0.67         1
              Adel_Al-Jubeir       0.00      0.00      0.00         1
       Adolfo_Aguilar_Zinser       0.00      0.00      0.00         1
                Adrien_Brody       1.00

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [6]:
import os
import random
from PIL import Image

lfw_root = os.path.join(extract_path, "lfw_funneled")


rand_idx = random.randint(0, len(X_test)-1)
true_person = y_test[rand_idx]


person_folder = os.path.join(lfw_root, true_person)
if not os.path.exists(person_folder):
  raise FileNotFoundError(f"❌ Person {true_person} not found in {lfw_root}")

img_name = random.choice(os.listdir(person_folder))
test_img_path = os.path.join(person_folder, img_name)


test_img = Image.open(test_img_path).convert("RGB")
test_emb = get_embedding(test_img).reshape(1, -1)

pred = classifier.predict(test_emb)[0]
proba = classifier.predict_proba(test_emb).max()

print("🖼️ Test image path:", test_img_path)
print("🖼️ True person:", true_person)
print("🔍 Predicted:", pred)



🖼️ Test image path: /kaggle/working/lfw_funneled/lfw_funneled/Colin_Powell/Colin_Powell_0039.jpg
🖼️ True person: Colin_Powell
🔍 Predicted: Colin_Powell


In [7]:
from google.colab import files
files.download("classifier.pkl")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>